In [1]:
'''
tcloud中用销售订单生成税局发票模板,分别按每个销售订单汇总，再按存货编码汇总，批量生成发票模板，存于同一文件夹下


'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd
import shutil

In [2]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型


In [3]:

path = r"F:\repos\fisha\莱新销售订单0826-0925"
os.chdir(path)

#以选择的销售订单所在文件夹作为工作目录，并在此目录下建“发票模板”文件夹
path1 = '发票模板'
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass

    


In [4]:
def getPivot(df):
    bianba_fenlei = dict(zip(df['存货编码'],df['存货分类']))
    bianba_mingchen = dict(zip(df['存货编码'],df['存货名称']))
    bianba_daima = dict(zip(df['存货编码'],df['存货代码']))
    bianba_hanlian = dict(zip(df['存货编码'],df['件含量']))
    pivot = df.pivot_table(index = '存货编码',values = ['数量','数量（件）','含税金额'] ,aggfunc = 'sum')
    pivot = pivot.reset_index()
    pivot = pivot.assign(fenlei = pivot['存货编码'].map(bianba_fenlei))
    pivot = pivot.assign(mingchen = pivot['存货编码'].map(bianba_mingchen))
    pivot = pivot.assign(daima = pivot['存货编码'].map(bianba_daima))
    pivot = pivot.assign(hanliang = pivot['存货编码'].map(bianba_hanlian))
    dic = dict(zip(['fenlei','mingchen','daima','hanliang'],['存货分类','存货名称','存货代码','件含量']))
    pivot = pivot.rename(columns =  dic)
    guige_qian = []
    guige_hou = []
    for i in pivot['存货名称'].to_list():
        qian0,hou0 = guige(i)
        guige_qian.append(qian0)
        guige_hou.append(hou0)
    pivot = pivot.assign(qian = guige_qian)
    pivot = pivot.assign(hou = guige_hou)
    return pivot

def getFapiaoBen(d):
    d['项目名称'] = d['存货编码'] +  d['hou']
    d['项目名称'] = d['项目名称'].str.replace('运费-','运费')
    # d['项目名称'] = d['项目名称'].str.split('-').str[0]
    d['商品和服务税收分类编码'] = '1060202010000000000'
    d['规格型号'] = d['qian']
    d['单位'] = '本'
    d['商品数量'] = d['数量']
    d['商品单价'] = ''
    d['金额'] = d['含税金额']
    d['税率'] = 0.13
    d['折扣金额'] = ''
    d['优惠政策类型'] = ''
    d = d[['项目名称',
     '商品和服务税收分类编码',
     '规格型号',
     '单位',
     '商品数量',
     '商品单价',
     '金额',
     '税率',
     '折扣金额',
     '优惠政策类型',]]
    return d

def getFapiaoMoban(gongsi):
    filename=''.join(['发票模板-',gongsi,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def getFapiaoMobanDingdanhao(gongsi,dingdanhao):
    filename=''.join(['发票模板-',gongsi,'-',dingdanhao,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def fengefu(string):
    num = len(string.split('-'))
    if num  == 3 :
        string = '-'.join([string.split('-')[0],string.split('-')[1]])
    elif num == 2 :
        if len(string.split('-')[0]) <= 4:
            string = string
        else :
            string = string.split('-')[0]
    else :
        string = string
    return string

def guige(string):
    if ('型' in string) and ('页' in string):
        qian0 = string.split('型')[0] + '型'
        hou0 =  string.split('型')[1] 
    elif ('型' in string) or ('页' in string):
        if '型' in string :
            qian0 = string.split('型')[0] + '型'
            hou0 =  string.split('型')[1] 
        else :
            qian0 = string.split('页')[0] + '页'
            hou0 =  string.split('页')[1] 
    else :
        qian0 = ''
        hou0 = ''
    return qian0,hou0    
            
    
    

In [5]:
#莱新销售订单超5000条，不能一次导出，分三次导出，并分别存于同一文件下，先将它们合并
data = []
for i in os.listdir(path):
    if  os.path.isfile(i):
        i = os.path.join(path,i)
        df = pd.read_excel(i)
        data.append(df)
    else :
        continue
   

df_xiaoshou0 = pd.concat(data)
df_xiaoshou1 = df_xiaoshou0.loc[df_xiaoshou0['单据执行状态'] != '合计']
lst1 = ['单据编号',
  '单据日期',
 '含税总金额',
 '存货名称',
 '存货分类',
 '存货编码',
  '存货代码',
  '数量',
  '件含量',
 '数量（件）',
  '含税单价',
 '含税金额',
        '客户']
df_xiaoshou2 = df_xiaoshou1[lst1]
df_xiaoshou3 = df_xiaoshou2.copy()
s = []
for i in df_xiaoshou2['存货编码'].to_list():
    j = fengefu(i)
    s.append(j)
df_xiaoshou3['存货编码'] = s

In [6]:
gp = df_xiaoshou3.groupby('单据编号')
for k,v  in gp:
    dingdanhao = k
    gongsi = v['客户'].to_list()[0]
    newname = getFapiaoMobanDingdanhao(gongsi,dingdanhao)
    print(newname)
    pivot = getPivot(v)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)
    
    
    
    
    
    
    
    
    
    


发票模板\发票模板-山东刘璐-SO-2024-02-0570.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


发票模板\发票模板-安康市申达文化用品有限公司L-SO-2024-02-0571.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


发票模板\发票模板-武汉市齐达利商贸有限公司-SO-2024-02-0572.xlsx


...

C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


发票模板\发票模板-芜湖迈斯广告传媒有限公司-SO-2024-02-1197.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


发票模板\发票模板-兴义市鸿图锐意办公商贸有限公司-SO-2024-02-1198.xlsx
发票模板\发票模板-遵义王强-SO-2024-02-1220.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
import pandas as pd
 
# 示例数据
df = pd.DataFrame({
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': ['small', 'large', 'large', 'small', 'small', 'large'],
     'F': ['small1', 'large1', 'large1', 'small1', 'small1', 'large1'],
    'D': [1, 2, 2, 3, 3, 4],
    'E': [2, 4, 5, 5, 6, 6]
})
 
df

In [ ]:
import pandas as pd
 
# 示例数据
df = pd.DataFrame({
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': ['small', 'large', 'large', 'small', 'small', 'large'],
    'D': [1, 2, 2, 3, 3, 4],
    'E': [2, 4, 5, 5, 6, 6]
})
 
# 创建透视表，并使用margins参数汇总
pivot_table = df.pivot_table(values='D', index=['A', 'B'], columns=['C'], aggfunc='sum', margins=True)
 
# 将多层索引转换为常规列，并将不是汇总的字段展开
melted = pivot_table.reset_index().melt(id_vars=['A', 'B'], value_vars=pivot_table.columns.levels[1].unique(), var_name='C')
 
# 将汇总行转换为常规列
melted['C'] = melted['C'].str.split('(').str[0]
 
# 重置索引
melted = melted.dropna(subset=['C']).reset_index(drop=True)
 
print(melted)

In [ ]:
import pandas as pd

# 创建示例数据集
data = {
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': [1, 2, 3, 4, 5, 6],
    'D': [7, 8, 9, 10, 11, 12]
}

df = pd.DataFrame(data)

# 创建透视表
pivot_table = pd.pivot_table(df, values='C', index='A', columns='B', aggfunc='sum', fill_value=0)
print(pivot_table)


In [ ]:
df = pd.read_clipboard()
df

In [ ]:
df['fengeshu'] = df['存货编码'].str.split('-').str.len()

In [ ]:
df['fengeshu'] 

In [ ]:
df.to_excel('fenge.xlsx',index = False)

In [ ]:

    
    
    
    # df['fengeshu'] = df['存货编码'].str.split('-').str.len()
    # df['bianma1'] = df['存货编码']
    # df = df.assign(bianma1  = np.where(df['fenggeshu'] == 1, df['存货编码'],np.where(df['fenggeshu'] == 2,)))
    
    

In [ ]:
s = []
for i in df['存货编码'].to_list():
    print(i)
    
    j = fengefu(i)
    print('j',j)
    s.append(j)
s


In [ ]:
s

In [ ]:
len(s)

In [ ]:
len(df['存货编码'])

In [ ]:
df

In [ ]:
df['存货编码1'] = s

In [ ]:
df.to_excel('fengehou.xlsx',index = False)

In [ ]:
df_xiaoshou3

In [ ]:
pivot

In [ ]:
df1 = pd.read_clipboard()
df1


In [ ]:
# guiges_qian = []
# guiges_hou = []
def guige(string):
    if ('型' in string) and ('页' in string):
        qian = string.split('型')[0] + '型'
        hou =  string.split('型')[1] 
    elif ('型' in string) or ('页' in string):
        if '型' in string :
            qian = string.split('型')[0] + '型'
            hou =  string.split('型')[1] 
        else :
            qian = string.split('页')[0] + '页'
            hou =  string.split('页')[1] 
    else :
        qian = ''
        hou = ''
                
  
    
    return qian,hou
    
        
    
        
    

In [ ]:
qian,hou = guige('aa型aa')
qian
hou

In [ ]:
"型" in 'aa型aa型'

In [ ]:
'aa型aa'.startswith('型') != True

In [ ]:
'型aa型aa型'.count('型')

In [ ]:
guiges_qian = []
guiges_hou = []
for i in df1['规格型号'].to_list():
    qian,hou = guige(i)
    print(qian,hou)
    guiges_qian.append(qian)
    guiges_hou.append(hou)


In [ ]:
guiges_qian

In [ ]:
guiges_hou

In [ ]:
len(guiges_hou)

In [ ]:
pivot

In [ ]:
qian,hou = guige('16K80精品道林页卡面语文本B2301')

In [ ]:
qian

In [ ]:
hou

In [ ]:
guige_qian = []
guige_hou = []
for i in pivot['存货名称'].to_list():
    qian0,hou0 = guige(pivot)
    guige_qian.append(qian0)
    guige_hou.append(hou0)
pivot = pivot.assign(qian = guige_qian)
pivot = pivot.assign(hou = guige_hou)
pivot

In [ ]:
guige_qian = []
guige_hou = []
for i in pivot['存货名称'].to_list():
    qian0,hou0 = guige(pivot)
    guige_qian.append(qian0)
    guige_hou.append(hou0)
pivot = pivot.assign(qian = guige_qian)
pivot = pivot.assign(hou = guige_hou)
pivot
    

In [ ]:
pivot